<h1 align="center">Space-Lunch</h1>
A journey through coding with the Engineering Process

![EDP](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.sciencebuddies.org%2Fblog%2Fgraphics%2F2013-blog-scientific-method-engineering-design-charts-750px.png&f=1&nofb=1)

# Defining our Problem

## Abstract
Look up places to eat near launch sites

## Objective
Create a Python Jupyter Notebook application that will allow a user to select upcoming SpaceX launches and then display places to eat back to the user on a map.

## Requirements
1. Allow user to select between upcoming SpaceX launches.
2. Display places to eat near selected launch back to user on a map.

## Project Steps:
1. [x] Get the list of upcoming launches including date/time and location. Data source: Probably SpaceX API
2. [x] Extract the relevant data (launchpad location, time of launch).
3. [x] Display the information to the user.
4. [x] Allow user to select between launches. (This gives us the launch location)
5. _ Use launch location to get the restaurants. Data Source: Probably Google API
6. _ Parse the restaurant data
7. _ Display the restaurants on a map.

## Research Data:
Most of our research will be on the data sources we intend to use, so we can link to the documentation for that here:

1. Data source for upcoming launches: https://api.spacexdata.com/v4/launches/upcoming
    * Launchpad value comes back as a very ugly ID number (i.e.: 5e9e4501f509094ba4566f84)
2. Data source to convert launchpad IDs to text: 
3. Data source for places to eat nearby: https://developers.google.com/maps/documentation/places/web-service/search-nearby
    * Required Parameter: Location - The point around which to retrieve place information. This must be specified as latitude,longitude.
    * As this requires our location be in lat/lng format, we will need to convert our launch locations to lat/lng.

# Prototyping
This section of the notebook is designated just for prototyping and testing out pieces of code. Much of our code is "test as you go", testing out each each step in the project as we program it, but this section set aside for all of our messy experimentation while we try to figure out what kind of approach we want to take to building each step.

In [1]:
# Import Dependencies
import requests
import pandas as pd
import gmaps
import matplotlib.pyplot as plt
import datetime
import time

In [2]:
resp = requests.get("https://api.spacexdata.com/v4/launches/upcoming").json()
resp

[{'fairings': {'reused': None,
   'recovery_attempt': None,
   'recovered': None,
   'ships': []},
  'links': {'patch': {'small': 'https://i.imgur.com/CaF1N0S.png',
    'large': 'https://i.imgur.com/XdcZC8w.png'},
   'reddit': {'campaign': 'https://www.reddit.com/r/spacex/comments/sarr7x/rspacex_csg2_campaign_thread/',
    'launch': 'https://www.reddit.com/r/spacex/comments/sdtz77/rspacex_csg2_launch_discussion_and_updates_thread/',
    'media': None,
    'recovery': None},
   'flickr': {'small': [], 'original': []},
   'presskit': None,
   'webcast': 'https://youtu.be/AbFoi68L-GQ',
   'youtube_id': 'AbFoi68L-GQ',
   'article': None,
   'wikipedia': None},
  'static_fire_date_utc': '2022-01-23T21:22:00.000Z',
  'static_fire_date_unix': 1642972920,
  'net': False,
  'window': None,
  'rocket': '5e9d0d95eda69973a809d1ec',
  'success': None,
  'failures': [],
  'details': 'Falcon 9 launches to sun-synchronous polar orbit from Florida as part of CSG-2 Mission. The mission lifts off from SL

In [3]:
date = resp[0]['date_local']
date

'2022-01-28T18:11:12-05:00'

In [4]:
pad = resp[0]['launchpad']
pad

'5e9e4501f509094ba4566f84'

In [5]:
# That launchpad ID is pretty gross, let's convert it to something we can use!
resp = requests.get("https://api.spacexdata.com/v4/launchpads/" + pad).json()
resp

{'images': {'large': ['https://i.imgur.com/9oEMXwa.png']},
 'name': 'CCSFS SLC 40',
 'full_name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
 'locality': 'Cape Canaveral',
 'region': 'Florida',
 'latitude': 28.5618571,
 'longitude': -80.577366,
 'launch_attempts': 79,
 'launch_successes': 77,
 'rockets': ['5e9d0d95eda69973a809d1ec'],
 'timezone': 'America/New_York',
 'launches': ['5eb87cddffd86e000604b32f',
  '5eb87cdeffd86e000604b330',
  '5eb87cdfffd86e000604b331',
  '5eb87ce0ffd86e000604b332',
  '5eb87ce1ffd86e000604b333',
  '5eb87ce2ffd86e000604b335',
  '5eb87ce3ffd86e000604b336',
  '5eb87ce4ffd86e000604b337',
  '5eb87ce4ffd86e000604b338',
  '5eb87ce5ffd86e000604b339',
  '5eb87ce6ffd86e000604b33a',
  '5eb87ce7ffd86e000604b33b',
  '5eb87ce8ffd86e000604b33c',
  '5eb87ceaffd86e000604b33d',
  '5eb87ceaffd86e000604b33e',
  '5eb87cecffd86e000604b33f',
  '5eb87cedffd86e000604b340',
  '5eb87ceeffd86e000604b341',
  '5eb87cefffd86e000604b342',
  '5eb87cf2ffd86e000604b344',


# Parsing SpaceX Data
Now that we are done prototyping, let's build a block of code to output just the date and location for each upcoming SpaceX launch.

In [6]:
# Get list of upcoming launches from SpaceX API
resp = requests.get("https://api.spacexdata.com/v4/launches/upcoming").json()

# Get date/time and location for each launch
launches = [] # Create a list to store launch information in

for launch in resp: # For each "launch" dictionary in the "resp" list of launches:
    
    date = launch['date_local'] # Retrieve the date for that launch
    pad_id = launch['launchpad']# Retrieve the pad ID for that launch
    
    # Convert the pad ID into a pad name using an additional API call
    resp2 = requests.get("https://api.spacexdata.com/v4/launchpads/" + pad_id).json()
    pad = resp2['full_name'] # Pull the full pad name out of the resp2 dictionary
    
    # Put the date and pad name for this launch in a dictionary and add that to the launches list
    launches.append({'date': date, "pad": pad})

# Convert the launches list of dictionaries to a DataFrame
df = pd.DataFrame(launches)

# Display the list of launches to the user
df

,date,pad
0,2022-01-28T18:11:12-05:00,Cape Canaveral Space Force Station Space Launc...
1,2022-01-30T14:39:00-05:00,Kennedy Space Center Historic Launch Complex 39A
2,2022-02-02T08:37:00-08:00,Vandenberg Space Force Base Space Launch Compl...
3,2022-01-31T19:00:00-05:00,Kennedy Space Center Historic Launch Complex 39A
4,2022-03-30T20:00:00-04:00,Kennedy Space Center Historic Launch Complex 39A
5,2022-03-31T17:00:00-07:00,Vandenberg Space Force Base Space Launch Compl...


# Get User Selection

In [7]:
# Get the user selection
selection = int(input("Select a launch to visit by the index number: "))
selection

Select a launch to visit by the index number:  1


1

In [8]:
# Based on user selection, get the appropriate pad location.
location = df['pad'][selection]

# Display that pad back to the user
location

'Kennedy Space Center Historic Launch Complex 39A'

# Pull and Parse Restaurant Data
After we have finished retrieving the SpaceX launch Data and asking the user what launch they wish to visit, we can move on to retrieving places to eat near that launch site.

In [9]:
# Import our gkey (Do not forget to make your own config.py)
from config import gkey

In [10]:
# Google geocoding API
# Now we can convert the pad name into lat/lng coordinates using the google geocoding API.
resp = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={gkey}").json()
resp

{'results': [{'address_components': [{'long_name': 'Florida',
     'short_name': 'FL',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Brevard County',
     'short_name': 'Brevard County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']}],
   'formatted_address': 'Kennedy Space Center, FL, USA',
   'geometry': {'location': {'lat': 28.6263826, 'lng': -80.6204703},
    'location_type': 'GEOMETRIC_CENTER',
    'viewport': {'northeast': {'lat': 28.6277315802915,
      'lng': -80.61912131970848},
     'southwest': {'lat': 28.6250336197085, 'lng': -80.6218192802915}}},
   'place_id': 'ChIJZer0yrO74IgRbTR4IzfGf1s',
   'plus_code': {'compound_code': 'J9GH+HR Wilson, FL, USA',
    'global_code': '76WXJ9GH+HR'},
   'types': ['establishment', 'museum', 'point_of_interest']}],
 'status': 'OK'}

In [ ]:
# google maps restaurant API call
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params_dict = {
    "location": f'{lat}, {lng}',
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey
}

In [17]:
base = "https://www.google.com/maps/@"
sat = "/data=!3m1!1e3"
lat = "28.6263826"
lng = "-80.6204703"
height = "4000"

url = f"{base}{lat},{lng},{height}m{sat}"
print(url)

https://www.google.com/maps/@28.6263826,-80.6204703,4000m/data=!3m1!1e3
